In [ ]:
# Setup - ensure we're using the local package
import sys
sys.path.insert(0, '../src')

from telescope_mcp import drivers
from telescope_mcp.drivers import DriverMode

# Use digital twin for testing without hardware
drivers.use_digital_twin()
print(f"Driver mode: {drivers.get_factory().config.mode}")

## Camera Driver Test

In [ ]:
# Get the camera driver from factory
factory = drivers.get_factory()
camera_driver = factory.create_camera_driver()

# List connected cameras
cameras = camera_driver.get_connected_cameras()
print("Connected cameras:")
for cam_id, info in cameras.items():
    name = info['Name'].decode() if isinstance(info['Name'], bytes) else info['Name']
    print(f"  [{cam_id}] {name} - {info['MaxWidth']}x{info['MaxHeight']}")

In [ ]:
# Open finder camera and capture a frame
finder = camera_driver.open(0)
print(f"Camera info: {finder.get_info()}")
print(f"Controls: {list(finder.get_controls().keys())}")

In [ ]:
# Capture and display a frame
import cv2
import numpy as np
from IPython.display import Image, display

jpeg_bytes = finder.capture(exposure_us=100000)
print(f"Captured {len(jpeg_bytes)} bytes")
display(Image(data=jpeg_bytes))

In [ ]:
# Test control setting
print(f"Current gain: {finder.get_control('ASI_GAIN')}")
finder.set_control('ASI_GAIN', 75)
print(f"After setting: {finder.get_control('ASI_GAIN')}")

## Motor Controller Test

In [ ]:
from telescope_mcp.drivers.motors import MotorType

motor_controller = factory.create_motor_controller()

# Check initial status
alt_status = motor_controller.get_status(MotorType.ALTITUDE)
az_status = motor_controller.get_status(MotorType.AZIMUTH)
print(f"Altitude: {alt_status}")
print(f"Azimuth: {az_status}")

In [ ]:
# Move motors
motor_controller.move(MotorType.ALTITUDE, steps=100, speed=50)
motor_controller.move(MotorType.AZIMUTH, steps=-50, speed=100)

# Check new position
print(f"Altitude after move: {motor_controller.get_status(MotorType.ALTITUDE)}")
print(f"Azimuth after move: {motor_controller.get_status(MotorType.AZIMUTH)}")

In [ ]:
# Home motors
motor_controller.home(MotorType.ALTITUDE)
motor_controller.home(MotorType.AZIMUTH)
print(f"After homing: Alt={motor_controller.get_status(MotorType.ALTITUDE).position_steps}, Az={motor_controller.get_status(MotorType.AZIMUTH).position_steps}")

## Position Sensor Test

In [ ]:
position_sensor = factory.create_position_sensor()

# Read position
pos = position_sensor.read()
print(f"Current position: Alt={pos.altitude}°, Az={pos.azimuth}°")

In [ ]:
# Calibrate to known position
position_sensor.calibrate(altitude=30.0, azimuth=270.0)
pos = position_sensor.read()
print(f"After calibration: Alt={pos.altitude}°, Az={pos.azimuth}°")

## Switching to Hardware Mode

When real hardware is connected, switch modes:

In [ ]:
# Uncomment to switch to hardware mode (requires real devices)
# drivers.use_hardware()
# factory = drivers.get_factory()
# camera_driver = factory.create_camera_driver()  # Would use real pyasi

print(f"Current mode: {drivers.get_factory().config.mode}")